In [1]:
import matplotlib.pyplot as plt, numpy as np
#%matplotlib inline

In [2]:
from astroML.plotting import scatter_contour 
from astroML.stats import median_sigmaG
from scipy.stats import binned_statistic
import seaborn as sns 
sns.set_context("poster")

In [3]:
# READ IN THE FILES  

def read_file(obj):
    '''
    By default, want to use SDSS_r filter data. However, accidentally 
    I saved SDSS_g filter data for Stars for Sample_4  . Sample_4 for QSO has 
    SDSS_r mags.   So  I made a Sample_6 , with identical criteria (hence 
    the same number of lines), but storing SDSS_r mag instead.  Thus, if need
    be,  for stars I can plot delflx / delflxerr vs SDSS_r or SDSS_g 
    
    '''


    if obj == 'StarB_g' : 
        File = 'Sample_4_Stars_blue_314248_lines.txt'
    if obj == 'StarB':
        File = 'Sample_6_Stars_blue_314248_lines.txt'
        
    if obj == 'StarR_g' :
        File = 'Sample_4_Stars_red_373306_lines.txt'
    if obj == 'StarR':
        File = 'Sample_6_Stars_red_373306_lines.txt'
        
    if obj == 'QSO' :
        File = 'Sample_6_QSO_721283_lines.txt'
        
    colnames = open(File,'r').read().splitlines()[0][1:].split()
    datatable = np.genfromtxt(File)
    data = {}
    print 'Zipping ', obj, ' data from ', File, ' ...'
    for label, column in zip(colnames, datatable.T):
        data[label] = column

    return data

In [4]:
# DEFINE GAUSSIAN TO BE CONVOLVED 

def gaussian(x,mu,sigma):
    exponent = -(x-mu)**2.0 / (2.0 * (sigma ** 2.0))
    f = (1.0 / (np.sqrt(2.0*np.pi)*sigma)) * np.exp(exponent)    
    return f

def gaussgauss_logL(xi, ei, mu, sigma):
    ''' Verbatim from AstroML  '''
    """Equation 5.63: gaussian likelihood with gaussian errors"""
    ndim = len(np.broadcast(sigma, mu).shape)

    xi = xi.reshape(xi.shape + tuple(ndim * [1]))
    ei = ei.reshape(ei.shape + tuple(ndim * [1]))

    s2_e2 = sigma ** 2 + ei ** 2
    return -0.5 * np.sum(np.log(s2_e2) + (xi - mu) ** 2 / s2_e2,  -1 - ndim)


In [5]:
def p_sigma_mu(xi, ei, mu_s, sig_s,  get_sigma = False, sig_lim=[0.00,0.5], mu_lim=[-0.2,0.2]):
    '''
    Instead of calculating approximate mu and sigma with approximate_mu_sigma(),
    I calculate their distribution p_sigma, p_mu, and choose the best value...
    The exact method, more time-consuming than the approximate_mu_sigma() but 
    better 
    '''
    # I assume sigma and mu range as I think they are for my distribution 
    sigma = np.linspace(sig_lim[0], sig_lim[1], sig_s)
    mu = np.linspace(mu_lim[0], mu_lim[1], mu_s)
    
    if get_sigma == False :
        logL = gaussgauss_logL(xi, ei, mu, sigma[:, np.newaxis])
        logL -= logL.max()
        L = np.exp(logL)
        
        p_sigma = L.sum(1)
        p_sigma /= (sigma[1] - sigma[0]) * p_sigma.sum()
        
        p_mu = L.sum(0)
        p_mu /= (mu[1] - mu[0]) * p_mu.sum()
        
        return p_mu, p_sigma
        
    if get_sigma == True :
        return mu , sigma
     
def approximate_mu_sigma(xi, ei, axis=None):
    """Estimates of mu0 and sigma0 via equations 5.67 - 5.68"""
    if axis is not None:
        xi = np.rollaxis(xi, axis)
        ei = np.rollaxis(ei, axis)
        axis = 0

    mu_approx, sigmaG = median_sigmaG(xi, axis=axis)
    e50 = np.median(ei, axis=axis)
    var_twiddle = (sigmaG ** 2 + ei ** 2 - e50 ** 2)
    sigma_twiddle = np.sqrt(np.maximum(0, var_twiddle))

    med = np.median(sigma_twiddle, axis=axis)
    mu = np.mean(sigma_twiddle, axis=axis)

    zeta = np.ones_like(mu)
    zeta[mu != 0] = med[mu != 0] / mu[mu != 0]

    var_approx = zeta ** 2 * sigmaG ** 2 - e50 ** 2
    sigma_approx = np.sqrt(np.maximum(0, var_approx))

    return mu_approx, sigma_approx     
     
def get_sigma_mu(xi,ei, return_sigma=False, return_p=False, y_34 = 'mode',  
                 mu_s=100, sig_s=40, sig_lim=[0.00,0.5], mu_lim=[-0.2,0.2],
                 approx=False):
    '''
    y_34 defines how sigma and mu are calculated based on p_sigma and p_mu 
    distributions : available   mode, exp, median (note : need to increase
    sigma and mu calculation grid to get median right !)
    
    return sigma : if set to True, it returns mu, sigma  linspaces. 
    return_p : if set to True, then it returns mu, sigma, as well as  p_mu, p_sigma 
               distributions
    mu_s : number specifying the number of grid points in mu space 
    sig_s : number specifying the number of grid points in sigma space 
    check_y34 : boolean variable (True or False), specifying whether 
    sig_lim : a list of two values: lower and upper limits on sigma linspace 
    mu_lim : a list of two values: lower and upper limits on mu linspace 
    
    '''
    #
    # RETURN ONLY SIGMA, MU LINSPACE ....
    #
    
    if return_sigma == True : 
        mu, sigma = p_sigma_mu(xi,ei, mu_s, sig_s, True, sig_lim, mu_lim)
        return mu, sigma
        
    # 
    # APPROXIMATE WAY 
    #
    if approx == True : 
        mu_i, sigma_i = approximate_mu_sigma(xi, ei)
        return mu_i, sigma_i
    
    #
    # EXACT WAY  
    #
    if approx == False : 
       
        p_mu,p_sigma = p_sigma_mu(xi,ei, mu_s, sig_s, False, sig_lim, mu_lim)
        # Change into numpy arrays 
        p_mu = np.array(p_mu)
        p_sigma = np.array(p_sigma)
        #  Retrieve sigma, mu linspace 
        mu, sigma = p_sigma_mu(xi,ei, mu_s, sig_s,  True, sig_lim, mu_lim )    
        
        # find sigma 1, 2 or 3 for this bin.... 
        
        if y_34 == 'mode' : 
            sig_max = sigma[p_sigma == max(p_sigma)][0]
            mu_max = mu[p_mu== max(p_mu)][0]
            sig_bin, mu_bin = sig_max, mu_max
            print  ' Here I am , and my sig_lim are ', sig_lim
        if y_34 == 'exp' :
            exp_val_sig = (sigma[1]-sigma[0])*np.sum(p_sigma*sigma)
            exp_val_mu = (mu[1]-mu[0])*np.sum(p_mu*mu)
            sig_bin, mu_bin = exp_val_sig, exp_val_mu
            
        if y_34 == 'median' : 
            
            # use only if enough gridding !  
            if sig_s > 100 : 
                delta_sig = sigma[1]-sigma[0]
                suma = 0
                i=0
                # integrating until the sum is 1/2 
                while suma < 0.5 : 
                    suma += delta_sig * p_sigma[i]
                    i += 1 
                sig3 = sigma[i-1]
                sig_bin = sig3
                
            if mu_s > 40 :
                
                delta_mu = mu[1]-mu[0]
                suma = 0
                i=0
                # integrating until the sum is 1/2 
                while suma < 0.5 : 
                    suma += delta_mu * p_mu[i]
                    i += 1 
                mu3 = mu[i-1]
                mu_bin = mu3
            else :
                print 'Too bad ! '
                print 'Your sigma gridding is too small to accurately calculate median....'
                print 'Calculating it with np.median instead of the proper definition '
                print ' (of integrating delta_sigma* p(sigma) until = 0.5 '
                
                sig_3 = np.median(p_sigma)
                mu_3 = np.median(p_mu)
                sig_bin, mu_bin = sig_3, mu_3
      
        # Two exit ways... 
        
        if return_p == True :
            return mu_bin, sig_bin, p_mu, p_sigma
            
        if return_p == False : 
            return mu_bin, sig_bin



In [8]:
# Calculate statistics for the sample ... 

def get_stats(y, y_err, mu_sig_sample):
    """ Calculate statistics for the y, y_err 1D arrays
    
    This function calls get_sigma_mu to calculate mu, sigma, based on the 
    p_sigma and p_mu distributions based on Fig. 5.7 AstroML.  
    Size of sigma  and mu space over which 
    p_sigma and p_mu are calculated is variable, depends on  mu_s, sig_s which 
    define the size of linear spaces, and  mu_lim, sig_lim , which define their
    extent.  From probability distributions both sigma and mu are calculated 
    as mean, mode or expectation value of the probability distribution, 
    depending on the value of y_34 parameter.  
    
    One can also use the approximate way of finding mu, sigma, but by default 
    it is disabled. 
    """     
    y_34    = mu_sig_sample['y_34']
    mu_s    = mu_sig_sample['mu_s']
    sig_s   = mu_sig_sample['sig_s']
    sig_lim = mu_sig_sample['sig_lim']
    mu_lim  = mu_sig_sample['mu_lim']
     
    mu_bin, sigma_bin = get_sigma_mu(y, y_err, y_34, sig_s=sig_s, mu_s=mu_s,
                                 sig_lim = sig_lim, mu_lim = mu_lim)
    #mu_app, sigma_app = approximate_mu_sigma(xi, ei)
    SF = sigma_bin
    #mu_mean = np.mean(y)    
        
    return SF, mu_bin 

       
def get_histogram(xdata, nbins):
    """ Calculate simple histogram given the 1D data array, and number of bins. 
    
    It first calculates unnormalized histogram (because numpy normalization 
    works in  a way so that the integral over all space=1 , which is not what
    I want). Then it divides it by the number of elements in the input data array,
    and the bin width. 
    In case a normalisation was needed, I calculate the area under the histogram,
    to scale up the gaussian by the area. 
    In case a normalised gaussian was needed, I also calculate it, but bins 
    are the same as for not-normalised gaussian . 
    """
    
    N_hist_ttl = float(len(xdata))
    hist, bin_edges = np.histogram(xdata, bins=nbins, density=False) 
    bin_cen = (bin_edges[:-1] + bin_edges[1:])/2 
    bin_width = (bin_edges[-1] - bin_edges[0]) / float(nbins)
    bin_width = bin_edges[1]-bin_edges[0]
    hist  = hist / (N_hist_ttl * bin_width)
    area = np.sum(bin_width * hist)
    
    print 'N_hist_ttl=', N_hist_ttl
    print 'Delta_bin = ', bin_width
    
    
    # Calculate normalised histogram, so that the INTEGRAL under the curve=1
    # This allows me to overplot the Gaussian too! 
    hist_n, edges = np.histogram(xdata, bins=nbins, density=True)
    return hist, hist_n, bin_cen, area        
    
    
def combine_gaussians(hist_xlim, sigma, mu, xi, ei, err_f=1.0 ):
    """ A function to convolve gaussians according to chosen model
    
    """
    
    xgrid = np.linspace(hist_xlim[0], hist_xlim[1], 10000)
    N = float(len(xi))
           
    
    model2 = np.zeros_like(xgrid)
    for i in range(len(xi)):
        sig_com = np.sqrt(sigma ** 2.0 + (err_f*ei[i])**2.0)
        Gi =  gaussian(xgrid,mu, sig_com)
        model2 += Gi  
    model2= (1.0 / N) * model2   
    
    return xgrid, model2
    
# PLOT THINGS...
# Cols are # delflx  delflxerr  obj_ID  SDSS_r_mMed
    
#starB_gMag = read_file('StarB_g')
#starB_rMag = read_file('StarB')
#starR_gMag = read_file('StarR_g') 
#starR_rMag = read_file('StarR')
QSO = read_file('QSO') 

#fig1 = plt.figure(figsize=(12,12))
#ax1 = fig1.add_subplot(221)
#
## Use SDSS g magnitudes...
##ax1.scatter( starB_gMag['SDSS_g_mMed'], starB_gMag['delflx'] / starB_gMag['delflxerr']  )
#scatter_contour(starB_gMag['SDSS_g_mMed'], starB_gMag['delflx'] / starB_gMag['delflxerr'],
#                threshold=400, log_counts=True, ax=ax1,histogram2d_args=dict(bins=40),
#                plot_args=dict(marker=',', linestyle='none', color='black'))
#ax1.set_xlabel('SDSS g magnitudes')
#ax1.set_ylabel(r'$\chi= \Delta (CRTS \, mag) \, / \,(CRTS \, err)$')
#ax1.set_title('Blue Stars')

# Use SDSS r magnitudes... 

def print_chi(data_dict, axis):
    """ Make the chi plot 
    
    Some more doc if need be ... 
    """

    mags = data_dict['SDSS_r_mMed'] 
    chi = data_dict['delflx'] / data_dict['delflxerr']
    
    scatter_contour(mags, chi, threshold=400, log_counts=True, ax=axis,
                    histogram2d_args=dict(bins=40), 
                    plot_args=dict(marker=',', linestyle='none', color='black'),
                    contour_args=dict(cmap=plt.cm.jet))
    axis.set_xlim(xmin=17)
    axis.set_ylim(ymin=-20, ymax=20)
    axis.set_xlabel('SDSS r mag' )
    axis.set_ylabel(r'$\chi = \Delta (CRTS \, mag) / (CRTS \, error) $')
    plt.savefig('test_test_test.png')

def load_xi_ei(data_dict, mag_min, mag_max):
    """ Load delflx, delflxerr from sample files (star or QSO)
    
    It assumes that the data dictionary has fields delflx, delflxerr,
    SDSS_r_mMed. So it'll not work if we use SDSS_g_mMed, or anything else...
    """
    
    mask = (data_dict['SDSS_r_mMed'] < mag_max) * (data_dict['SDSS_r_mMed'] >mag_min)
    delflx = data_dict['delflx'][mask]
    delflxerr = data_dict['delflxerr'][mask]
    return delflx, delflxerr
    
    
# PLOT  HISTOGRAMS and GAUSSIANS for r magnitudes.... 

# SETTINGS THAT CONTROL THE APPEARANCE OF HISTOGRAM, AND SIGMA, MU LINSPACES 
mu_sig_sample={}
colnames = ['bins_hist', 'hist_xlim', 'mu_s', 'sig_s','mu_lim', 'sig_lim','y_34', 'approx']
datatable = [200, [-1.5,1.5], 40, 100, [-0.007,0.007 ], [0.17, 0.19],'mode', True]

for label, column in zip(colnames, datatable):
    mu_sig_sample[label] = column  
    
    
# BLUE Stars   mag: 17-18 
#delflx, delflxerr = load_xi_ei(starB_rMag, 17, 18)

# QSO

def sci_notation(num, decimal_digits=1, precision=None, exponent=None):
    """
    Returns a string representation of the scientific
    notation of the given number formatted for use with
    LaTeX or Mathtext, with specified number of significant
    decimal digits and precision (number of decimal digits
    to show). The exponent to be used can also be specified
    explicitly.
    
    From 
    """
    if not exponent:
        exponent = int(floor(log10(abs(num))))
    coeff = round(num / float(10**exponent), decimal_digits)
    if not precision:
        precision = decimal_digits

    return r"${0:.{2}f}\cdot10^{{{1:d}}}$".format(coeff, exponent, precision)



def plot_p_distr(delflx, delflxerr):
    mu_lim = [-0.004,-0.001]
    sig_lim = [0.149, 0.153]
    
    mu,sigma = get_sigma_mu(xi=delflx, ei=delflxerr, sig_lim=sig_lim, 
                            mu_lim=mu_lim, return_sigma=True)
    mu_bin, sig_bin, p_mu, p_sigma = get_sigma_mu(xi=delflx, ei=delflxerr, 
                            sig_lim=sig_lim, mu_lim=mu_lim, return_p=True)
    fig = plt.figure()
    plt.clf()
   
    ax1 = fig.add_subplot(211)
    ax1.plot(mu, p_mu, '-o')
    ax1.set_title('p(mu)')
    ax1.scatter(mu_bin, p_mu[np.abs(mu-mu_bin) < 0.00001], marker='o', c='y')
    
    ax2 = fig.add_subplot(212)
    ax2.plot(sigma, p_sigma, '-o')
    ax2.set_title('p(sigma)')
    fig.tight_layout()
    plt.savefig('p_sigma_p_mu_QSO_18.5-19.png')
    plt.show()    

    print 'Calculating mode of p_sigma , p_mu, we get sigma=',sig_bin, \
    ' mu=', mu_bin
    
    return mu, sigma, p_mu, p_sigma, mu_bin, sig_bin

#out= plot_p_distr(delflx, delflxerr)
 #SF, mu = get_stats(delflx, delflxerr, mu_sig_sample) 


mag_min = [17, 18, 18.5]
mag_max = [18, 18.5, 19]
qso_sig = [0.0997, 0.10456, 0.15105]
qso_mu = [-0.00097, -0.00033, -0.00254]
hist_qso = [-1.0,1.0]
#starB_sig
#starB_mu

fig, axs = plt.subplots(2,2, figsize=(12, 12), facecolor='w', edgecolor='k')
#fig.subplots_adjust(hspace = .5, wspace=.5)
axs = axs.ravel()
# from http://stackoverflow.com/questions/17210646/python-subplot-within-a-loop-first-panel-appears-in-wrong-position

# First plot the chi distr in the upper left corner 
print_chi(data_dict=QSO, axis=axs[0])

# Then loop over different ranges and plot the models... 
for i in range(len(mag_min)):
    print 'Populating plot ', i+1
    delflx, delflxerr = load_xi_ei(QSO, mag_min[i], mag_max[i])
    delflx = delflx[(delflx < hist_qso[1] )*(delflx>hist_qso[0])]
    
    hist, hist_n, bins, area = get_histogram(xdata=delflx, nbins=150)
    
    # first model : SF^2 + ( 1.0 * err) ^2
    
                                 
    # second model : SF^2 + ( 1.3 * err) ^2
    xgrid, model2 = combine_gaussians(xi=delflx, ei=delflxerr, 
                                 hist_xlim=hist_qso, sigma=qso_sig[i], 
                                 mu=qso_mu[i], err_f=1.3 )                             
                                 
    # Plot the histogram    
    axs[i+1].plot(bins, hist,ls='steps', lw=2, label='data')
    axs[i+1].set_xlabel(r'$\Delta m$')
    axs[i+1].set_ylabel(r'$n(bin) / (N \cdot \Delta (bin))$')
    # Plot  model1
    model1_lab = r'$\sigma=%d, \, \mu=%d, \, f_{c}=1.0$' % (qso_sig[i], qso_mu[i])
    axs[i+1].plot(xgrid, model1, lw=2, label=model1_lab )
    
    # Plot model2
    model2_lab = r'$\sigma=%d, \, \mu=%d, \, f_{c}=1.3$' % (qso_sig[i], qso_mu[i])
    axs[i+1].plot(xgrid, model2, lw=2, label=model2_lab )

    # Set x limits
    axs[i+1].set_xlim(xmin=hist_qso[0], xmax=hist_qso[1])
    
    # Get y limits  
    ymin, ymax = axs[i+1].get_ylim()
    
    # Add text saying the limits ... 
    axs[i+1].text(0.25, 0.75*ymax, r'$%.1f < m < %.1f$'%(mag_min[i], mag_max[i]), fontsize=20)   
    

Zipping  QSO  data from  Sample_6_QSO_721283_lines.txt  ...
Populating plot  1
N_hist_ttl= 32331.0
Delta_bin =  0.013192
Populating plot  2
N_hist_ttl= 55109.0
Delta_bin =  0.0132193333333
Populating plot  3
N_hist_ttl= 121242.0
Delta_bin =  0.0133246666667


In [ ]:
lgd =  plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))   
plt.savefig('QSO_sample_panels.png',
            bbox_extra_artists=(lgd,), bbox_inches='tight')
plt.show()

In [ ]:
import this

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(xgrid, model1)
ax.plot(xgrid,model2)
ymin, ymax = ax.get_ylim()
ax.text(0.25, 0.75*ymax, r'$%.1f < m < %.1f$'%(mag_min[i], mag_max[i]), fontsize=30)

In [ ]:
# Check models... 

xgrid, model1 = combine_gaussians(xi=delflx, ei=delflxerr, 
                                 hist_xlim=hist_qso, sigma=qso_sig[i], 
                                 mu=qso_mu[i], err_f=1.0 )